In [1]:
import torch
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from visdom import Visdom
%run Utility_Functions.ipynb
%run parameters.py

In [2]:
# Data 
x = loadmat('data.mat')['x']
y = loadmat('data.mat')['y']

viz = Visdom()
# Device configuration 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device # verify which device is being used

device(type='cuda', index=0)

In [3]:
# Prepare the data for convolution neural network 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

class custom_dataset(Dataset):
    def __init__(self, x, y):
        super(custom_dataset, self).__init__()
        self.x = torch.from_numpy(x).type(torch.float32)
        self.y = torch.from_numpy(y).type(torch.float32)
    
    def __getitem__(self, ids):
        return self.x[ids], self.y[ids] # 2 index of total force ir index
    
    def __len__(self):
        return self.x.shape[0]

# Create datasets 
train_data = custom_dataset(x_train, y_train)
test_data = custom_dataset(x_test, y_test)


def get_loss(data):
    with torch.no_grad(): 
        out = model(data.x.to(device))
        y_true = data.y.to(device)
        error = torch.sqrt(criterion(out, y_true))
        error = error.cpu().detach()

        return error

In [4]:
# Hyper-parameters 
input_size = x.shape[1]
hidden_size = 5
output_size = y.shape[1]
num_epochs = 500
batch_size = 32
learning_rate = 0.001

# Load datasets 
train_data_iterator = DataLoader(train_data, batch_size=batch_size, shuffle=True)

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.input = torch.nn.Linear(input_size, hidden_size)
        self.output = torch.nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out = self.input(x)
        out = torch.tanh(out)
        out = self.output(out)
        return torch.tanh(out)

def get_loss(data):
    with torch.no_grad(): 
        out = model(data.x.to(device))
        y_true = data.y.to(device)
        error = torch.sqrt(criterion(out, y_true))
        error = error.cpu().detach()
        return error
    
    
model = NeuralNet(input_size, hidden_size, output_size).to(device)
viz.close(win='Accuracy')

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

i = 0
for epoch in range(num_epochs):
    for x_batch, y_batch in train_data_iterator:
        # Send the images and labels to gpu 
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        # Forward pass
        out_put = model(x_batch) 
        loss = torch.sqrt(criterion(out_put, y_batch))
        
        # Backward and optimize
        optimizer.zero_grad() # For batch gradient optimisation
        loss.backward()
        optimizer.step()
        
    # Calculate the accuracy
    test_accuracy = get_loss(test_data)
    train_accuracy = get_loss(train_data)

    i = i + 1 
    viz.line(X = np.column_stack([i]*2), Y = np.column_stack((train_accuracy, test_accuracy)), win='Accuracy', update='append')